In [4]:
from geodesic.tesseract import Job, AssetSpec, Step, Container, StepInput, StepOutput, TemporalBinning, GlobalProperties, User
from geodesic import Dataset
from geodesic.account import Credential
import geodesic
import numpy as np

In [48]:
c = Credential.from_gcp_service_account(
    name='gcp-storage'
)

In [49]:
c.create()

In [50]:
geodesic.account.get_credentials()

[{'uid': '25d1446eb6b3483eb93c4bd92d858293a9b80d7e',
  'name': 's3cred',
  'type': 'AWS_KEY_PAIR'},
 {'uid': '46647980ca8439f038c5bdaed9cf5e24399e6fa6',
  'name': 'gcp-storage',
  'type': 'SERVICE_ACCOUNT_KEY'}]

In [7]:
p = geodesic.create_project(
    name='paci-demo',
    alias='paci demo',
    description='A demo project for Paci data and the Esri models running in tesseract.',
    keywords='paci, Esri, road, DL'
)
geodesic.set_active_project('paci-demo')

{'name': 'paci-demo',
 'alias': 'paci demo',
 'description': 'A demo project for Paci data and the Esri models running in tesseract.',
 'owner': 'auth0|60a83a706acb9a00718c598a',
 'keywords': 'paci, Esri, road, DL',
 'uid': '477024689b4e8a33cb61767fba1d116fc163c9c2'}

In [51]:
paci = Dataset.from_bucket(
    name="paci-imagery",
    url="gs://geodesic-test-data/paci_test_data",
    oriented=False,
    credential="gcp-storage",
    description="High resolution imagery from Paci."
)
paci.save()

In [52]:
ds = geodesic.list_datasets(search='paci')
ds

{'paci-imagery': dataset:*:*:*:paci-imagery}

In [55]:
paci = ds[0]
q = paci.query()
q

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': 'paci2',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[5338624.640677031, 3416253.333070929],
      [5341384.558037605, 3416253.333070929],
      [5341384.558037605, 3414227.162396702],
      [5338624.640677031, 3414227.162396702],
      [5338624.640677031, 3416253.333070929]]]},
   'bbox': (5338624.640677031,
    3414227.162396702,
    5341384.558037605,
    3416253.333070929),
   'properties': {},
   'assets': {'image': {'href': 'gs://geodesic-test-data/paci_test_data/paci2.tif',
     'title': 'Image',
     'description': 'image paci2 from a cloud bucket',
     'type': 'image/tiff',
     'roles': ['dataset'],
     'eo:bands': [{'name': 'band_0',
       'description': 'unknown raster band',
       'gsd': 0.5971441081546087},
      {'name': 'band_1',
       'description': 'unknown raster band',
       'gsd': 0.5971441081546087},
      {'name': 'band_2',
       'description': 'unknown raster band',
       'gsd': 0.5971441081546087},
      {'name': 'band_3',
       'description': 'unknown raster band',
       'gsd': 0.5971441081546087}]}}}],
 'links': [],
 'numberMatched': 0,
 'numberReturned': 0,
 'timestamp': '2022-08-01T22:04:21.676211987Z'}

In [57]:
bbox = q.features[0].bbox

(5338624.640677031, 3414227.162396702, 5341384.558037605, 3416253.333070929)

In [62]:
dict(q.features[0])

{'type': 'Feature',
 'id': 'paci2',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[5338624.640677031, 3416253.333070929],
    [5341384.558037605, 3416253.333070929],
    [5341384.558037605, 3414227.162396702],
    [5338624.640677031, 3414227.162396702],
    [5338624.640677031, 3416253.333070929]]]},
 'bbox': (5338624.640677031,
  3414227.162396702,
  5341384.558037605,
  3416253.333070929),
 'properties': {},
 'assets': {'image': {'href': 'gs://geodesic-test-data/paci_test_data/paci2.tif',
   'title': 'Image',
   'description': 'image paci2 from a cloud bucket',
   'type': 'image/tiff',
   'roles': ['dataset'],
   'eo:bands': [{'name': 'band_0',
     'description': 'unknown raster band',
     'gsd': 0.5971441081546087},
    {'name': 'band_1',
     'description': 'unknown raster band',
     'gsd': 0.5971441081546087},
    {'name': 'band_2',
     'description': 'unknown raster band',
     'gsd': 0.5971441081546087},
    {'name': 'band_3',
     'description': 'unknown raster band',


In [ ]:
job = Job(
    alias="Paci Road Model Test",
    description=f"Running the Esri DLPK Road Model on data from Paci. The Paci data for now is a single geotiff in an AWS bucket using a Boson dataset.",
    global_properties=GlobalProperties(
        pixel_size=[0.5971441081546087, 0.5971441081546087],                           
        chip_size=2000
    ),
    bbox=bbox,                                       
    bbox_epsg=3857,                                 
    output_epsg=3857,
    asset_specs=[
        AssetSpec(
            name="paci-imagery",                       
            dataset="paci-imagery",           
            assets=['band_0', 'band_1', 'band_2'],                
            pixel_type=np.float32,                   
            input_no_data=['nan'],                                  
        )
    ],
    steps=[
        Step(
            name="road-model",
            type="model",
            container=Container(
                repository="gcr.io/double-catfish-291717",
                image="esri-road-model",
                tag="v0.0.1"
            ),
            gpu=False,
            inputs=[
                StepInput(
                    asset_name="paci-imagery",
                    chunk_shape=(12, 1, 39, 36),
                    type="tensor",
                )
            ],
            outputs=[
                StepOutput(
                    asset_name="roads",
                    chunk_shape=(12, 1, 39, 36),
                    type="tensor",
                    pixel_dtype=np.float32,
                    fill_value="nan"
                )
            ]
        ),
    ],
    workers=1
)